# Weights and biases have-a-go

We're going to learn how to use weights and biases with the aid of everyone's favourite, the [Kaggle Titanic dataset](https://www.kaggle.com/competitions/titanic/data).

In [ ]:
from dotenv import load_dotenv
import logging
import pandas as pd
import os
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
import wandb

from utils import PROJECT_DIR, load_data, SEED

load_dotenv()

os.chdir(PROJECT_DIR)

WANDB_PROJ = "Titanic"  # Change this to your project name!
WANDB_USER = os.environ.get("wandb_username")  # Change this to your username!
JOB = "predict survival"

## 1. Log a run in Weights and Biases

First things first, let's load the data:

In [ ]:
X_train, X_test, y_train, y_test = load_data()

We are going to train a classifier to predict whether a passenger survived the Titanic disaster. We'll start simple with a logistic regression model.

The first step in using weights and biases is to initialise a **run**. Just as it sounds, a run captures a record of each time you *run* your model. You can choose what metrics, visuals etc you want to record in each run.

The way you start a run is like this:

In [ ]:
run = wandb.init(
        project=WANDB_PROJ,
        job_type=JOB,
        save_code=True,
    )

Note:
- I log both `project` and `job_type` because within one project, you might have multiple ML jobs that you want to keep track of. For example, at Nesta, a real use case might be an imputation model to predict and fill in missing values, then another downstream model to make predictions based on the imputed data.
- `save_code`: definitely do this! It saves a snapshot of your code at the time of the run. This is mega useful because you might not want to make a Git commit every single time you run your code - you'll do lots of minor changes and tweaking.

The run now exists as an object called `run`.

If you go to your weights and biases workspace, find the Titanic project and navigate to **Runs**, you should now see that you have a run in progress! Exciting!

We will now define and fit a logistic regression model. First of all, we'll define a dict of hyperparameters. The wandb guidance notes that you should keep track of ALL hyperparameters, even if you're just using the defaults. I've been a bit lazy and haven't exactly done that here, but you should!

The hyperparameters are stored in a dict because that will make it more convenient when we move on to running sweeps.

In [ ]:
log_reg_config = {'penalty': 'l2',
                  'C': 1.0,
                  'random_state': SEED,
                  'solver':'lbfgs',
                  'max_iter':100}
    
model = LogisticRegression(penalty=log_reg_config['penalty'],
                           C=log_reg_config['C'],
                           solver=log_reg_config['solver'],
                           max_iter=log_reg_config['max_iter'],
                           random_state=log_reg_config['random_state'])

model.fit(X_train, y_train)

Let's find out how the model did:

In [ ]:
preds = model.predict(X_test)
accuracy = accuracy_score(y_test, preds)
logging.info(f"Accuracy: {accuracy}")

It got 81% accuracy which is not bad for a first go. We will definitely want to store this metric with the run so that we can compare accuracy across different runs. As a bonus, we'll also create and log a confusion matrix.

The way you log things to the run is extremely simple for single metrics, and not much more complicated for artefacts like tables and graphs.

In [ ]:
wandb.run.summary["accuracy"] = accuracy

cm = confusion_matrix(y_test, preds)
cm = pd.DataFrame(cm)
logging.info(f"Confusion matrix:\n{cm}")

# Log confusion matrix
wb_confusion_matrix = wandb.Table(data=cm, columns=["0", "1"])
run.log({"confusion_matrix": wb_confusion_matrix})

Now we're done, so we'll finish the run! You can go to the **Runs** page and inspect :)

In [ ]:
wandb.finish()

## 2. Sweep over hyperparameters

Ok, we have a model. It got only 81% accuracy, so we might want to try some different hyperparameters to see if we can improve it. Weights and biases [sweeps](https://docs.wandb.ai/guides/sweeps) are a really convenient way to do this.

To run a sweep, we need:
- a config that contains: `method` ie the search method; `metric` ie the metric to optimise for; `parameters` ie values of the different hyperparameters to try
- a functions that define the training procedure
- a sweep agent that runs the sweep

You can search the hyperparameter space using either Bayesian search, grid search or random search. Random search is quick and efficient and is the approach we'll take here. For a random search, you need to tell weights and biases how many random possible combinations of hyperparameters to try.

The code below sets the number of runs and the sweep config, and repackages the code we used for model training earlier as a `train()` function:

In [ ]:
N_RUNS = 5 # How many different hyperparameter combinations to try aka how many different runs

sweep_configuration = {
    'method': 'random',  # grid, random, or bayes. Random picks random combinations of hyperparameter values. Grid picks all possible combinations. Bayes picks based on past results.
    'metric': {
      'name': 'accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'penalty': {'values': ['l2', None]},
        'solver': {'values': ['lbfgs']}, # even if there are hyperparameters that you aren't planning on varying, it is still good to record these!
        'C': {
            'values': [0.01, 0.1, 1]
        },
        'max_iter': {
            'values': [10, 100, 1000]
        }
    }
}

def train(config, X_train, X_test, y_train, y_test):
    # Build the model
    model = LogisticRegression(C=config['C'],
                               max_iter=config['max_iter'],
                               penalty = config['penalty'],
                               random_state=SEED,
                               solver = config['solver'])
    model.fit(X_train, y_train)

    # Predict and evaluate
    preds = model.predict(X_test)
    accuracy = accuracy_score(y_test, preds)
    
    cm = confusion_matrix(y_test, preds)
    cm = pd.DataFrame(cm)
    
    return accuracy, cm

We now define a function `main()` that contains everything we want to happen within a single run:

In [ ]:
def main():
    wandb.init(project=WANDB_PROJ,
               job_type=JOB,
               save_code=True,
               tags=["logistic regression"])
    X_train, X_test, y_train, y_test = load_data()
    accuracy, cm = train(wandb.config,  X_train, X_test, y_train, y_test)
    wandb.log({"accuracy": accuracy})
    
    # Log confusion matrix
    wb_confusion_matrix = wandb.Table(data=cm, columns=["0", "1"])
    wandb.log({"confusion_matrix": wb_confusion_matrix})
    wandb.finish()

We now have everything we need to kick off a sweep!

In [ ]:
sweep_id = wandb.sweep(sweep=sweep_configuration, project=WANDB_PROJ)

wandb.agent(sweep_id, entity=WANDB_USER, function=main,
                count=N_RUNS
                )

You can now navigate back to your weights and biases Titanic project and navigate to **Sweeps**. You should see the details of the sweep we just ran. The sweep info will tell you some useful things like:
- Which run within the sweep got the best accuracy
- Parameter importance: which hyperparameters contributed most to the model's performance. This might inform you whether to try more runs at smaller increments of those hyperparameters.

**Bonus:** within a sweep you can not just try different hyperparameters, but you can also try out different models or data preprocessing steps, embeddings models etc. You can see an example of how to do this in the script `sweep_different_classifiers.py`. There is guidance on how to nest parameters in your sweep config [here](https://docs.wandb.ai/guides/sweeps/define-sweep-configuration).

That's all for now :) See more detailed info and a handy colab notebook on sweeps [here](https://docs.wandb.ai/guides/sweeps).